In [171]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [172]:
car_dataframe = pd.read_csv("data/car details v4.csv")
data = (car_dataframe.loc[:, ["Fuel Tank Capacity", "Year", "Price"]]).to_numpy().astype('float64')
data_clean = data[~np.isnan(data).any(axis=1)]
print(data_clean)
x = data_clean[:, :2]
y = data_clean[:, 2:]
for i in range(x.shape[1]):
    x[:, i:i+1] = (x[:, i:i+1] - x[:, i:i+1].mean()) / x[:, i:i+1].std()
y = y / y.mean()
print(x)
print(y)

[[3.500e+01 2.017e+03 5.050e+05]
 [4.200e+01 2.014e+03 4.500e+05]
 [3.500e+01 2.011e+03 2.200e+05]
 ...
 [3.200e+01 2.014e+03 2.750e+05]
 [4.500e+01 2.013e+03 2.400e+05]
 [6.500e+01 2.018e+03 4.290e+06]]
[[-1.12550346  0.13375241]
 [-0.66212109 -0.79032635]
 [-1.12550346 -1.71440511]
 ...
 [-1.3240959  -0.79032635]
 [-0.46352864 -1.09835261]
 [ 0.86042098  0.44177866]]
[[0.30088322]
 [0.26811376]
 [0.13107784]
 ...
 [0.1638473 ]
 [0.14299401]
 [2.55601784]]


In [173]:
def make_pred(x, w, b):
    return np.matmul(x, w) + b

def compute_cost(x, y, w, b):
    sqr_diff = (make_pred(x, w, b) - y)**2
    return sqr_diff.sum() / (2*x.shape[1])

def run_grad_decent(x, y, w, b, alpha):
    cost_sum = ((make_pred(x, w, b)-y).sum() / x.shape[1])
    new_b = b - alpha * cost_sum

    j = x.shape[1]
    i = x.shape[0]
    new_w = np.empty(j)
    for k in range(i):
        pred = w.reshape(-1).dot(x[k]) + b - y[k]
        for l in range(j):
            new_w[l] += (pred*x[k][l])[0]
    new_w /= j
    new_w = w - (alpha * new_w.reshape([j, 1]))
    return new_w, new_b

def run_linear_regression(x, y, alpha = .0001, iter = 2000):
    j = x.shape[1]
    w = np.empty([j, 1])
    b = 0
    print(compute_cost(x, y, w, b))
    for _ in range(iter):
        w, b = run_grad_decent(x, y, w, b, alpha)
        print(compute_cost(x, y, w, b))
    return w, b

In [174]:
run_linear_regression(x, y)

1476.3012173916443
3.157130061599118e+64
2.6085445576731483e+64
2.1550208734177835e+64
1.780347173747552e+64
1.4708145513424034e+64
1.2150975249882954e+64
1.0038396709395655e+64
8.293112809704411e+63
6.851265402782031e+63
5.6600987706847716e+63
4.67602934793599e+63
3.863051043562157e+63
3.1914178151498973e+63
2.636555291659915e+63
2.1781616223927094e+63
1.7994646530919083e+63
1.4866082500205661e+63
1.2281453182377881e+63
1.0146189641343179e+63
8.382164773938252e+62
6.924834719346927e+62
5.720877265425355e+62
4.726240843643886e+62
3.9045327273712943e+62
3.2256874593296195e+62
2.664866787345774e+62
2.201550858177211e+62
1.8187874171257517e+62
1.5025715424234318e+62
1.2413332195076606e+62
1.0255140060539096e+62
8.472172983736443e+61
6.999194027836654e+61
5.782308403445638e+61
4.776991513534642e+61
3.946459844096853e+61
3.2603250930929275e+61
2.693482293643869e+61
2.2251912490391197e+61
1.8383176702088858e+61
1.5187062496589988e+61
1.2546627332865814e+61
1.0365260396153717e+61
8.5631477073

(array([[0.80731853],
        [0.40562638]]),
 0.9999999999999977)